In [1]:
from graphviz import *
from sklearn.datasets import load_iris
from sklearn import tree
 
from collections import Counter
from itertools import chain, combinations
 
import ast
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import graphviz
import seaborn as sns
from sklearn import datasets, svm, metrics
from pandas import DataFrame
from sklearn.svm import SVR

In [2]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras import optimizers
from pandas import Series
import math
import numpy
from tensorflow.keras.layers import *
from tensorflow.keras import layers
import pandas as pd
from math import *
from sklearn.metrics import *

data = pd.read_csv("C:\\Users\\mozzimerci\\jupyter notebook\\time\\btc\\bitcoin2000.csv")

In [3]:
data

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,timeUTC
0,1576144800,7219.93,7149.15,7170.18,1536.79,11041994.81,7166.14,direct,NaN,2019-12-12 19:00:00
1,1576148400,7208.36,7166.03,7166.14,785.11,5654607.43,7199.79,direct,NaN,2019-12-12 20:00:00
2,1576152000,7222.43,7194.87,7199.79,672.46,4855213.22,7199.97,direct,NaN,2019-12-12 21:00:00
3,1576155600,7211.80,7187.29,7199.97,807.61,5821087.81,7201.38,direct,NaN,2019-12-12 22:00:00
4,1576159200,7229.22,7152.84,7201.38,1606.06,11537799.57,7173.50,direct,NaN,2019-12-12 23:00:00
...,...,...,...,...,...,...,...,...,...,...
10000,1612144800,33847.46,33257.39,33450.01,2066.49,69445570.35,33675.45,direct,NaN,2021-02-01 11:00:00
10001,1612148400,33779.76,33505.50,33675.45,1173.82,39473368.04,33590.20,direct,NaN,2021-02-01 12:00:00
10002,1612152000,33945.41,33404.87,33590.20,1430.06,48159838.72,33576.82,direct,NaN,2021-02-01 13:00:00
10003,1612155600,33907.89,33467.87,33576.82,1198.20,40319458.23,33803.02,direct,NaN,2021-02-01 14:00:00


In [4]:
def gradient_importance(seq, model):

    seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(seq)

    grads = tape.gradient(predictions, seq)
    grads = tf.reduce_mean(grads, axis=1).numpy()[0]
    
    return grads

def gradient_weight(seq, model):

    seq = tf.Variable(seq[np.newaxis,:,:], dtype=tf.float32)

    with tf.GradientTape() as tape:
        predictions = model(seq)

    grad = tape.gradient(predictions, seq)
    #gr=grads
    #grads = tf.reduce_mean(grads, axis=1).numpy()[0]
    
    return grad


def activation_grad(seq, model):
    
    seq = seq[np.newaxis,:,:]
    grad_model = Model([model.inputs], 
                       [model.get_layer('extractor').output, 
                        model.output])

    # Obtain the predicted value and the intermediate filters
    with tf.GradientTape() as tape:
        seq_outputs, predictions = grad_model(seq)

    # Extract filters and gradients
    output = seq_outputs[0]
    grads = tape.gradient(predictions, seq_outputs)[0]

    # Average gradients spatially
    weights = tf.reduce_mean(grads, axis=0)
    
    # Get a ponderated map of filters according to grad importance
    cam = np.ones(output.shape[0], dtype=np.float32)
    for index, w in enumerate(weights):
        cam += w * output[:, index]

    time = int(seq.shape[1]/output.shape[0])
    cam = zoom(cam.numpy(), time, order=1)
    heatmap = (cam - cam.min())/(cam.max() - cam.min())
    
    return heatmap

In [5]:
data.set_index(['timeUTC'],inplace=True)
#data = data['item_cnt_day'].resample('D').sum()
df=pd.DataFrame(data)

In [6]:
data

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
timeUTC,,,,,,,,,
2019-12-12 19:00:00,1576144800,7219.93,7149.15,7170.18,1536.79,11041994.81,7166.14,direct,NaN
2019-12-12 20:00:00,1576148400,7208.36,7166.03,7166.14,785.11,5654607.43,7199.79,direct,NaN
2019-12-12 21:00:00,1576152000,7222.43,7194.87,7199.79,672.46,4855213.22,7199.97,direct,NaN
2019-12-12 22:00:00,1576155600,7211.80,7187.29,7199.97,807.61,5821087.81,7201.38,direct,NaN
2019-12-12 23:00:00,1576159200,7229.22,7152.84,7201.38,1606.06,11537799.57,7173.50,direct,NaN
...,...,...,...,...,...,...,...,...,...
2021-02-01 11:00:00,1612144800,33847.46,33257.39,33450.01,2066.49,69445570.35,33675.45,direct,NaN
2021-02-01 12:00:00,1612148400,33779.76,33505.50,33675.45,1173.82,39473368.04,33590.20,direct,NaN
2021-02-01 13:00:00,1612152000,33945.41,33404.87,33590.20,1430.06,48159838.72,33576.82,direct,NaN


In [7]:
data.columns[1:7]

Index(['high', 'low', 'open', 'volumefrom', 'volumeto', 'close'], dtype='object')

In [8]:
from sklearn.preprocessing import MinMaxScaler
values = data[data.columns[1:7]].values.reshape(-1,6)
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
#scaled = (values-(np.min(values)))/(np.max(values)-(np.min(values)))

In [9]:
values.min(axis=0)

array([4.6234502e+03, 3.9489199e+03, 4.2406899e+03, 1.6186000e+02,
       1.1648342e+06, 4.2406899e+03], dtype=float32)

In [10]:
np.max(values)

7859574300.0

In [11]:
train_size = int(len(scaled) * 0.80)
val_size = int(len(scaled) * 0.10)
test_size = len(scaled) - train_size-val_size

In [12]:
train, val, test = scaled[0:train_size,:],scaled[train_size:train_size+val_size,:], scaled[train_size+val_size:,:]
print(len(train),len(val), len(test))

8004 1000 1001


In [13]:
train_x = train[:,1:]
train_y = train[:,0]
val_x = val[:,1:]
val_y = val[:,0]
test_x = test[:,1:]
test_y = test[:,0]

In [14]:
class CharacterTable(object):
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [15]:
X_train_min, X_train_max = train_x.min(axis=0), train_x.max(axis=0)
y_train_min, y_train_max = train_y.min(axis=0), train_y.max(axis=0)

In [16]:
train_x = (train_x - X_train_min)/(X_train_max - X_train_min+ 1e-9 )
val_x= (val_x - X_train_min)/(X_train_max - X_train_min+ 1e-9 )
test_x = (test_x - X_train_min)/(X_train_max - X_train_min+ 1e-9 )
train_y = (train_y - y_train_min)/(y_train_max - y_train_min + 1e-9)
val_y = (val_y - y_train_min)/(y_train_max - y_train_min+ 1e-9 )
test_y = (test_y - y_train_min)/(y_train_max - y_train_min+ 1e-9 )

## RMSE

### kernel = linear

In [17]:
# C는 학습 오류에 대한 패널티 C 값이 클 수록 모델이 학습 데이터에 좀 더 최적화 됨.
# 너무 크면 오버피팅 발생
# Epsilon 은 임계점을 표현 즉, 예측한 값이 GT 범위 안에 있으면 패널티 부여 X

# c가 커질수록 무조건 좋아짐?
c_1_list = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
rmse_list = []

# Validation
for c_1 in c_1_list:
    svr = SVR(kernel='linear', C=c_1, epsilon=0.1)
    svr.fit(train_x, train_y)

    yhat = svr.predict(val_x)
    rmse = sqrt(mean_squared_error(val_y, yhat))
    rmse_list.append(rmse)
    
    result = list(map(list, zip(c_1_list, rmse_list)))
    result_df = DataFrame(result,columns=['c', 'rmse'])

print('-------Validation-------')
print(result_df)

-------Validation-------
       c      rmse
0    0.1  0.150827
1    1.0  0.136516
2    2.0  0.136516
3    3.0  0.136516
4    4.0  0.136516
5    5.0  0.136516
6    6.0  0.136516
7    7.0  0.136516
8    8.0  0.136516
9    9.0  0.136516
10  10.0  0.136516


In [18]:
# Test
best_c_1 = result_df.iloc[result_df['rmse'].argmin()]['c']
svr = SVR(kernel='linear', C=best_c_1, epsilon=0.1) 
svr.fit(train_x, train_y)
predict = svr.predict(test_x)
test_rmse = sqrt(mean_squared_error(test_y, predict))

print('Test RMSE :', test_rmse)

Test RMSE : 0.39239483466725755


### kernel = RBF 
c

In [19]:
c_1_list = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
rmse_list = []

# Validation
for c_1 in c_1_list:
    svr = SVR(kernel='rbf', C=c_1, gamma = 1,  epsilon=0.1)
    svr.fit(train_x, train_y)

    yhat = svr.predict(val_x)
    rmse = sqrt(mean_squared_error(val_y, yhat))
    rmse_list.append(rmse)
    
    result = list(map(list, zip(c_1_list, rmse_list)))
    result_df = DataFrame(result,columns=['c', 'rmse'])

print('-------Validation-------')
print(result_df)

-------Validation-------
       c      rmse
0    0.1  0.490795
1    1.0  0.486367
2    2.0  0.486367
3    3.0  0.486367
4    4.0  0.486367
5    5.0  0.486367
6    6.0  0.486367
7    7.0  0.486367
8    8.0  0.486367
9    9.0  0.486367
10  10.0  0.486367


In [20]:
# Test
best_c_1 = result_df.iloc[result_df['rmse'].argmin()]['c']
svr = svr = SVR(kernel='rbf', C=c_1, gamma = 1,  epsilon=0.1)
svr.fit(train_x, train_y)
predict = svr.predict(test_x)
test_rmse = sqrt(mean_squared_error(test_y, predict))

print('Test RMSE :', test_rmse)

Test RMSE : 2.006535611194728


### kernel = RBF 
gamma

In [21]:
gmm_list = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9 ,10, 100]
rmse_list = []

# Validation
for gmm in gmm_list:
    svr = SVR(kernel='rbf', C=10, gamma = gmm,  epsilon=0.1)
    svr.fit(train_x, train_y)

    yhat = svr.predict(val_x)
    rmse = sqrt(mean_squared_error(val_y, yhat))
    rmse_list.append(rmse)
    
    result = list(map(list, zip(gmm_list, rmse_list)))
    result_df = DataFrame(result,columns=['gamma', 'rmse'])

print('-------Validation-------')
print(result_df)

-------Validation-------
    gamma      rmse
0     0.1  0.220922
1     1.0  0.486367
2     2.0  0.550987
3     3.0  0.591570
4     4.0  0.620055
5     5.0  0.643434
6     6.0  0.662464
7     7.0  0.679882
8     8.0  0.694203
9     9.0  0.705128
10   10.0  0.714239
11  100.0  0.812011


In [22]:
# Test
best_gmm = result_df.iloc[result_df['rmse'].argmin()]['gamma']
svr = svr = SVR(kernel='rbf', C=10, gamma = best_gmm,  epsilon=0.1)
svr.fit(train_x, train_y)
predict = svr.predict(test_x)
test_rmse = sqrt(mean_squared_error(test_y, predict))

print('Test RMSE :', test_rmse)

Test RMSE : 1.4997827398827255


## MAE

### kernel = linear

In [23]:
c_1_list = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
mae_list = []

# Validation
for c_1 in c_1_list:
    svr = SVR(kernel='linear', C=c_1, epsilon=0.1)
    svr.fit(train_x, train_y)

    yhat = svr.predict(val_x)
    mae = mean_absolute_error(val_y, yhat)
    mae_list.append(mae)
    
    result = list(map(list, zip(c_1_list, mae_list)))
    result_df = DataFrame(result,columns=['c', 'mae'])

print('-------Validation-------')
print(result_df)

-------Validation-------
       c       mae
0    0.1  0.145816
1    1.0  0.131663
2    2.0  0.131663
3    3.0  0.131663
4    4.0  0.131663
5    5.0  0.131663
6    6.0  0.131663
7    7.0  0.131663
8    8.0  0.131663
9    9.0  0.131663
10  10.0  0.131663


In [24]:
# Test
best_c_1 = result_df.iloc[result_df['mae'].argmin()]['c']
svr = SVR(kernel='linear', C=best_c_1, epsilon=0.1) 
svr.fit(train_x, train_y)
predict = svr.predict(test_x)
test_mae = mean_absolute_error(val_y, yhat)

print('Test MAE :', test_mae)

Test MAE : 0.13166275933489088


### kernel = RBF 
c

In [25]:
c_1_list = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
rmse_list = []

# Validation
for c_1 in c_1_list:
    svr = SVR(kernel='rbf', C=c_1, gamma = 1,  epsilon=0.1)
    svr.fit(train_x, train_y)

    yhat = svr.predict(val_x)
    mae = mean_absolute_error(val_y, yhat)
    mae_list.append(mae)
    
    result = list(map(list, zip(c_1_list, mae_list)))
    result_df = DataFrame(result,columns=['c', 'mae'])

print('-------Validation-------')
print(result_df)

-------Validation-------
       c       mae
0    0.1  0.145816
1    1.0  0.131663
2    2.0  0.131663
3    3.0  0.131663
4    4.0  0.131663
5    5.0  0.131663
6    6.0  0.131663
7    7.0  0.131663
8    8.0  0.131663
9    9.0  0.131663
10  10.0  0.131663


In [26]:
# Test
best_c_1 = result_df.iloc[result_df['mae'].argmin()]['c']
svr = svr = SVR(kernel='rbf', C=c_1, gamma = 1,  epsilon=0.1)
svr.fit(train_x, train_y)
predict = svr.predict(test_x)
test_mae = mean_absolute_error(val_y, yhat)

print('Test mae :', test_mae)

Test mae : 0.4137902366316792


### kernel = RBF 
gamma

In [27]:
gmm_list = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9 ,10, 100]
rmse_list = []

# Validation
for gmm in gmm_list:
    svr = SVR(kernel='rbf', C=10, gamma = gmm,  epsilon=0.1)
    svr.fit(train_x, train_y)

    yhat = svr.predict(val_x)
    mae = mean_absolute_error(val_y, yhat)
    mae_list.append(mae)
    
    result = list(map(list, zip(gmm_list, mae_list)))
    result_df = DataFrame(result,columns=['gamma', 'mae'])

print('-------Validation-------')
print(result_df)

-------Validation-------
    gamma       mae
0     0.1  0.145816
1     1.0  0.131663
2     2.0  0.131663
3     3.0  0.131663
4     4.0  0.131663
5     5.0  0.131663
6     6.0  0.131663
7     7.0  0.131663
8     8.0  0.131663
9     9.0  0.131663
10   10.0  0.131663
11  100.0  0.420814


In [28]:
# Test
best_gmm = result_df.iloc[result_df['mae'].argmin()]['gamma']
svr = svr = SVR(kernel='rbf', C=10, gamma = best_gmm,  epsilon=0.1)
svr.fit(train_x, train_y)
predict = svr.predict(test_x)
test_mae = mean_absolute_error(val_y, yhat)

print('Test mae :', test_mae)

Test mae : 0.7586128738898389
